In [1]:
import re
import pickle 
import os 
import glob
from tqdm import tqdm
import pympi.Elan 
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch 
from transformers import BertModel 
from transformers import BertJapaneseTokenizer

import utils

from transformers import logging
logging.set_verbosity_error()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
feature_path = '../data/Hazumi_features/'

# オークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking"
)

# 学習済みモデルの読み込み
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking").to(device)


def preprocess(text):
    """ 
    フィラー記号(F *)，|の除去
    """
    text = re.sub('\([^)]*\)', '', text)
    text = text.replace('|', '')

    return text


def embedding(sentences):
    # 前処理

    sentences = [preprocess(text) for text in sentences]

    # BERTトークン化
    encoded = tokenizer.batch_encode_plus(
        sentences, padding=True, add_special_tokens=True
    )


    # BERTトークンID列を抽出
    input_ids = torch.tensor(encoded["input_ids"], device=device) 

    # BERTの最大許容トークン数が512なので超える場合は切り詰める
    input_ids = input_ids[:, :512] 

    with torch.no_grad():
        outputs = model(input_ids) 

    # 最終層の隠れ状態ベクトルを取得
    last_hidden_states = outputs[0] 

    # [CLS]トークンの単語ベクトルを抽出
    vecs = last_hidden_states[:, 0, :]

    return vecs.tolist()


def eaf_to_df( eaf: pympi.Elan.Eaf ) -> pd.DataFrame:
    tier_names = list( eaf.tiers.keys() )

    def timeslotid_to_time( timeslotid: str ) -> float:
        return eaf.timeslots[ timeslotid ] / 1000

    def parse( tier_name: str, tier: dict ) -> pd.DataFrame:
        values = [ (key,) + value[:-1] for key, value in tier.items() ]
        df = pd.DataFrame( values, columns=[ "id", "start", "end", "transcription"] )

        df["start"] = df["start"].apply( timeslotid_to_time )
        df["end"] = df["end"].apply( timeslotid_to_time )
        df["ID"] = df.apply( lambda x: f"{tier_name}-{x.name}", axis=1 )
        df = df.reindex( columns=["ID", "start", "end", "transcription"] )

        return df

    dfs = [ parse(tier_name=name, tier=eaf.tiers[name][0]) for name in tier_names ]
    df = pd.concat( dfs )
    df = df.sort_values( "start" )
    df = df.reset_index( drop=True )
    return df

def extract_sentence(version, filename, start):
    res = []
    src = f'../data/Hazumi/Hazumi{version}/elan/' + filename + '.eaf' 

    eaf = pympi.Elan.Eaf(src) 
    df = eaf_to_df(eaf) 

    df['start'] = (df['start'] * 1000).astype(int)

    for time in start:
        sentence = df[(df['start'] == time) & (df['ID'].str.contains('user'))]['transcription'].values.tolist()
        if len(sentence) == 0:
            sentence = ['']
        sentence = embedding(sentence)
        res.append(sentence[0])

    return res


def personality_binarization(x, vid):
    """
    性格特性スコアを2クラスに分類
    """
    binary = {}
    df = pd.DataFrame.from_dict(x, orient='index')

    df = (df >= 0) * 1

    for id in vid:
        binary[id] = df.loc[id, :].tolist()

    return binary

def get_personality(hazumiversion, filename):
    """
    アノテータの質問回答から性格特性スコアを算出
    """
    if hazumiversion == "1911":
        filepath = "../data/Hazumi/Hazumi1911/questionnaire/1911questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=4, header=1, index_col=0)
    elif hazumiversion == "2010":
        filepath = "../data/Hazumi/Hazumi2010/questionnaire/questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=3, header=1, index_col=0)
    elif hazumiversion == "2012":
        filepath = "../data/Hazumi/Hazumi2012/questionnaire/questionnaires.xlsx"
        df = pd.read_excel(filepath, sheet_name=3, header=1, index_col=0)
    data = df.loc[filename].values.tolist()
    return [data[0] + (8 - data[5]), (8-data[1]) + data[6], data[2] + (8-data[7]), data[3] + (8-data[8]), data[4] + (8-data[9])]

def get_third_personality(version, filename):
    """
    アノテータの質問回答から性格特性スコアを算出
    """
    if version == "1911":
        file_path = '../data/Hazumi/Hazumi1911/questionnaire/220818thirdbigfive-Hazumi1911.xlsx'
    elif version == "2010":
        file_path = '../data/Hazumi/Hazumi2010/questionnaire/220822thirdbigfive-Hazumi2010.xlsx'
    df = pd.read_excel(file_path, sheet_name=5, header=1, index_col=0)
    data = df.loc[filename].values.tolist()
    return [data[5], data[13], data[21], data[29], data[37]]

/home/ryoyanagimoto/anaconda3/envs/personality/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1911
version = "1911"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

bert_text = {}
audio = {}
visual = {} 

bio = {}

SP = {}
TP = {}

# SS = {} 
SS_ternary = {}
# TS = {}
TS_ternary = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)[ms]'].values.tolist()

    vid.append(filename)
    bert_text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    bio[filename]  = df.loc[:, 'EDA_mean':'HR_std'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = get_third_personality(version, filename)

    # SS[filename] = df.loc[:, 'SS'].astype(int).values.tolist()
    SS_ternary[filename] = df.loc[:, 'SS_ternary'].astype(int).values.tolist()
    # TS[filename] = df.loc[:, 'TS1':'TS5'].mean(axis='columns').values.tolist()
    TS_ternary[filename] = df.loc[:, 'TS_ternary'].astype(int).values.tolist()
    

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)

# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS_ternary, TS_ternary, SP, TP, bert_text, audio, visual, bio, vid), f)

100%|██████████| 26/26 [00:21<00:00,  1.23it/s]


NameError: name 'SS_ternary' is not defined

In [20]:
# 2010
version = "2010"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

bert_text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)'].values.tolist()

    vid.append(filename)
    bert_text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis\n'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)
    TP[filename] = get_third_personality(version, filename)

    TS[filename] = df.loc[:, 'TS1':'TS3'].mean(axis='columns').values.tolist()
    

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)

# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, bert_text, audio, visual, vid), f)

100%|██████████| 33/33 [00:21<00:00,  1.56it/s]


In [4]:
version = "2012"
dumpfile_path = f'../data/Hazumi/Hazumi{version}/dumpfiles/*'
files = glob.glob(dumpfile_path)

bert_text = {}
audio = {}
visual = {} 

SP = {}
TP = {}

SS = {}
TS = {}

vid = []

for file_path in tqdm(sorted(files)):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)
    start = df['start(exchange)'].values.tolist()

    vid.append(filename)
    bert_text[filename] = extract_sentence(version, filename, start)
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis\n'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    SP[filename] = get_personality(version, filename)

    TS[filename] = df.loc[:, 'TS1':'TS3'].mean(axis='columns').values.tolist()
    

# SP_scaled = utils.dict_standardize(SP, vid)
# SP_binary = personality_binarization(SP_scaled, vid)

# SP_cluster = utils.clustering(SP_scaled, vid, n_clusters=4)

with open(feature_path + f'Hazumi{version}_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, bert_text, audio, visual, vid), f)

100%|██████████| 63/63 [00:40<00:00,  1.55it/s]


In [5]:
# All
with open('../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    SS_1911, TS_1911, SP_1911, TP_1911, text_1911, audio_1911, visual_1911, vid_1911 = pickle.load(f, encoding='utf-8')

with open('../data/Hazumi_features/Hazumi2010_features.pkl', mode='rb') as f:
    SS_2010, TS_2010, SP_2010, TP_2010, text_2010, audio_2010, visual_2010, vid_2010 = pickle.load(f, encoding='utf-8')

with open('../data/Hazumi_features/Hazumi2012_features.pkl', mode='rb') as f:
    SS_2012, TS_2012, SP_2012, TP_2012, text_2012, audio_2012, visual_2012, vid_2012 = pickle.load(f, encoding='utf-8')

def get_updatedict(Xs):
    res = {} 
    for x in Xs:
        res.update(x)
    return res

def get_extendlist(Xs):
    res = []
    for x in Xs:
        res.extend(x)
    return res

SS = get_updatedict([SS_1911, SS_2010, SS_2012])
TS = get_updatedict([TS_1911, TS_2010, TS_2012])

SP = get_updatedict([SP_1911, SP_2010, SP_2012])
TP = get_updatedict([TP_1911, TP_2010, TP_2012])

text = get_updatedict([text_1911, text_2010, text_2012])
audio = get_updatedict([audio_1911, audio_2010, audio_2012])
visual = get_updatedict([visual_1911, visual_2010, visual_2012])

vid = get_extendlist([vid_1911, vid_2010, vid_2012])

with open(feature_path + 'Hazumiall_features.pkl', mode='wb') as f:
    pickle.dump((SS, TS, SP, TP, text, audio, visual, vid), f)
